In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import sys
sys.path.append('/content/drive/MyDrive/Colab Notebooks')

In [ ]:
%matplotlib inline
from copy import deepcopy
from matplotlib import pyplot as plt

import imp
try:
    imp.find_module('jupyterplot')
    from jupyterplot import ProgressPlot
except ImportError:
    !pip install jupyterplot
    from jupyterplot import ProgressPlot

import torch
from torch import nn

from utils import train_step, test_step
from utils import BaselineModel
from utils import get_cifar10_dataset, make_dataloader

# Learning rate
![](https://mblogthumb-phinf.pstatic.net/MjAxOTA0MTlfMTg2/MDAxNTU1NjM0NTM2NjQw.ReuAZtSQXIgyGA98iiI_222dHBOMcjMRrew68NU3USUg.NLr7L3nuWQvE4WHGIUoELeQo2nnhghrObFAJVVB-MDQg.PNG.ollehw/image.png?type=w800)

- 여러가지 학습률을 모델이 적용해 훈련이 어떻게 되는지 확인해보기

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
num_epochs = 5
batch_size = 64
learning_rates = [1., 0.1, 0.01, 0.001]
momentum = 0.9
phases = ['train', 'test']

In [ ]:
cifar10_dataset = get_cifar10_dataset()
loader = make_dataloader(cifar10_dataset, batch_size)

In [ ]:
base_cnn = BaselineModel().to(device)

nets = [BaselineModel().to(device) for _ in learning_rates]
_ = [net.load_state_dict(deepcopy(base_cnn.state_dict())) for net in nets]

optimizers = [
    torch.optim.SGD(net.parameters(), lr, 0.9)
    for net, lr in zip(nets, learning_rates)
]
criterion = nn.CrossEntropyLoss()

pp = ProgressPlot(
    plot_names=phases,
    line_names=list(map(str, learning_rates)),
    x_lim=[0, num_epochs*len(loader['train'])],
    x_label='Iteration',
    y_lim=[[0, 3], [0, 100]]
)

accs = [0] * len(learning_rates)
for epoch in range(num_epochs):
    for inputs, target in loader['train']:
        losses = [
            train_step(net, inputs, target, optimizer, criterion, device)
            for net, optimizer in zip(nets, optimizers)
        ]
        pp.update([losses, accs])
    
    corrects = [0] * len(learning_rates)
    for inputs, target in loader['test']:
        outputs = [
            test_step(net, inputs, target, device=device)[0]
            for net in nets
        ]
        corrects = [
            (correct + (output.argmax(1).cpu() == target).sum()).item()
            for correct, output in zip(corrects, outputs)
        ]
        
    accs = [
        correct / len(cifar10_dataset['test']) * 100
        for correct in corrects
    ]
    
    print(f'Epoch: {epoch+1} accuracy ', end='')
    for lr, acc in zip(learning_rates, accs):
        print(f'{lr}: {acc:.2f}', end=' ')
    print()
pp.finalize()